In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Some parts of this Codelab are (c) Google 2025 under the Apache License.
(c) Aquarc 2025

# Synopsis
Aquarc is an all-in-one SAT platform for high schoolers designed to minimize time spent using the software and maximizing practice and essential questions. In order to further this mission, Aquarc Intelligence was created to analyze mistakes within a question and to suggest similar questions for efficient practicing.

# Install the SDK 
We will be using Google's Gemini and utilities to build the model.

In [2]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0" langchain PyPDF2 chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 21.8 MB/s eta 0:

Import the SDK and set up the API key

In [3]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

Set up a retry helper so we can press "Run All" and not worry about hitting the quota. 

In [4]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [5]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

### Choose your model
Depending on what's available and your quota, choose a model that's effective for your purposes.

In [6]:
client = genai.Client(api_key=GOOGLE_API_KEY)

for model in client.models.list():
  print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01

Check out the detailed information for your model

In [7]:
for model in client.models.list():
  if model.name == 'models/gemini-2.0-flash':
    print(model.to_json_dict())
    break

# change this line if you want to use a different model
model = "gemini-2.0-flash"

{'name': 'models/gemini-2.0-flash', 'display_name': 'Gemini 2.0 Flash', 'description': 'Gemini 2.0 Flash', 'version': '2.0', 'tuned_model_info': {}, 'input_token_limit': 1048576, 'output_token_limit': 8192, 'supported_actions': ['generateContent', 'countTokens']}


Test your model

In [8]:
chat = client.chats.create(model=model, history=[])
response = chat.send_message('Hello! My name is Zlork.')
print(response.text)

Nice to meet you, Zlork! It's a pleasure to make your acquaintance. Is there anything I can help you with today?



You can use the `Markdown()` function to format it nicely in Kaggle.

In [9]:
response = chat.send_message('Hello! My name is Zlork.' + 
                             'Use some fancy markdown in your message')
Markdown(response.text)

Ah, greetings, Zlork! It is an *uncommon* delight to be communicating with you! I hope this message finds you well.

***

Perhaps I can assist you with something *intriguing* today? Perhaps a poem in iambic pentameter? Or a complex query about the esoteric properties of, say, **dodecahedrons**?

```python
def say_hello_fancy(name):
  """Greets a person with markdown flair."""
  return f"## Greetings, {name}!\n\nIt is a *singular* pleasure to meet you.\n\n***\n\nMay your day be filled with wonder and punctuated with moments of exquisite delight!"

print(say_hello_fancy("Zlork"))
```

Let me know! What manner of assistance can I render?


# A fine prompt
Let's start by writing our prompt. What should it include and what should it mention? Following the principles taught throughout the course, let's place an importance on giving **positive** instructions rather than **negative** instructions to maintain effectiveness. 

The model needs to do the following:
- Stick to the SAT: Understand weighting and importance of certain questions, categories, and ensure all advice is applicable to the bounds of the SAT. A document with the specifications of the SAT format can be used for Retrieval Augmented Generation (RAG) rather than potential hallucination over the exact requirements.
- Have access to the current question (and maybe the previous questions for even more context)
- Have access to the answers and time between each to estimate confidence.
- Understand images or SVGs for Inference and other Reading/Writing questions.
- Use Tree of Thoughts (ToT) to generate multiple solving processes because multiple methods may be used to arrive at the same answer, and output these answering mechanisms in a JSON array to present on the website effectively (a TUI for this notebook)
- Find semantically related questions and present them to the user on the website using ReAct (a TUI for this notebook)

Let's start by finding an effective prompt. 
Then we can evaluate the effectiveness of including the SAT standards in a PDF to help the user answering a question. We will also evaluate whether a vector search database is useful for this document.

Below, the variables for the specific question we are testing are defined.

In [10]:
# All questions are (c) CollegeBoard 2025.
question = """
In a paper about p-i-n planar perovskite solar cells (one of several perovskite cell architectures designed to collect and store solar power), Lyndsey McMillon-Brown et al. described a method for fabricating the cell’s electronic transport layer (ETL) using a spray coating. Conventional ETL fabrication is accomplished using a solution of nanoparticles. The process can result in a loss of up to 80% of the solution, increasing the cost of manufacturing at scale—an issue that may be obviated by spray coating fabrication, which the researchers describe as “highly reproducible, concise, and practical.”

What does the text most strongly suggest about conventional ETL fabrication?
A. It is less suitable for manufacturing large volumes of planar p-i-n perovskite solar cells than an alternative fabrication method may be.
B. It is more expensive when manufacturing at scale than are processes for fabricating ETLs used in other perovskite solar cell architectures.
C. It typically entails a greater loss of nanoparticle solution than do other established approaches for ETL fabrication.
D. It is somewhat imprecise and therefore limits the potential effectiveness of p-i-n planar perovskite solar cells at capturing and storing solar power.
"""

rationale = """
Choice A is the best answer. Conventional solar cell fabrication increases “the cost of manufacturing at scale,” but spray coating might get rid of that problem.

Choice B is incorrect. This is not completely supported by the text. While it’s true that conventional ETL fabrication is expensive at scale, there’s nothing in the text that mentions other perovskite solar cell architectures. Choice C is incorrect. This choice does not match the text. Only one conventional method of ETL fabrication is described, so we can’t compare the solution loss in this method to that of other conventional methods. Choice D is incorrect. This choice isn’t supported by the text. The text never suggests that the effectiveness of solar cells changes based on their method of fabrication. 
"""

user_answer = "C"

Ideally, the user will talk to the chatbot after it gets the question wrong (or before in some scenarios, too). Either way, the user will have some rationale as to his or her answer to the question. Don't expect this rationale to be well thought out - the objective of the intelligent agent is to draw out what they actually mean. It may be completely omitted as well.

In [11]:
user_rationale = "Isn't the new method of ETL fabrication the same as the 'established methods'"

Synthesize the prompt:

In [12]:
from google.genai.types import GenerateContentConfig

response = client.models.generate_content(
    model="gemini-2.0-flash-001",
    # no specs yet
    contents=f"{question} {rationale}\n The user got: {user_answer}\n {user_rationale}",
    config=GenerateContentConfig(
        system_instruction=[
            "You are an SAT expert tutor. Analyze questions using the official SAT framework. ",
            "Help students by:\n",
            "1. Identifying question type and skills tested\n",
            "2. Explaining why answers are correct/incorrect\n",
            "3. Providing actionable improvement strategies\n",
            "Use formal but friendly language. Reference the SAT specs when possible.",
        ]
    ),
)


Markdown(response.text)


Okay, I can definitely understand why you chose option C and how you interpreted the text. Let's break down the question and the answer choices according to the College Board's SAT framework, and hopefully, this will clarify why A is the stronger answer.

**Question Type and Skills Tested:**

*   **Question Type:** This is a Command of Evidence question, specifically an Inference question. (Reading Test Specifications)
*   **Skills Tested:** You need to be able to understand what the author is suggesting, even if it's not stated directly. You have to draw a logical conclusion based on the information given.

**Analysis of the Text and Answer Choices:**

The key part of the text is: "...conventional ETL fabrication is accomplished using a solution of nanoparticles. The process can result in a loss of up to 80% of the solution, increasing the cost of manufacturing at scale—an issue that may be obviated by spray coating fabrication..."

*   **A. It is less suitable for manufacturing large volumes of planar p-i-n perovskite solar cells than an alternative fabrication method may be.**

    *   **Why it's correct:** The passage directly states that the conventional method increases "the cost of manufacturing *at scale*." The spray coating method "may obviate" this issue, implying it *is* more suitable for large-scale manufacturing. "Less suitable" is a good way to describe the problem in the original method.
*   **B. It is more expensive when manufacturing at scale than are processes for fabricating ETLs used in other perovskite solar cell architectures.**

    *   **Why it's incorrect:** The passage *only* discusses p-i-n planar perovskite solar cells. It makes *no comparison* to other perovskite cell architectures. This introduces information not found in the text.
*   **C. It typically entails a greater loss of nanoparticle solution than do other established approaches for ETL fabrication.**

    *   **Why it's incorrect:** This is where the misunderstanding lies. The passage describes *one* conventional method using nanoparticles. The phrase "established approaches" implies multiple conventional methods exist, and that *this* particular method loses more solution than *other* conventional methods. However, the text doesn't provide information about other established approaches or their solution loss.
*   **D. It is somewhat imprecise and therefore limits the potential effectiveness of p-i-n planar perovskite solar cells at capturing and storing solar power.**

    *   **Why it's incorrect:** The text focuses on *cost* and *reproducibility*. It doesn't mention the precision of conventional ETL fabrication or its impact on the solar cell's effectiveness.

**Why Your Initial Thought Was Understandable:**

You're right to focus on the idea of "established methods." However, "conventional ETL fabrication" in the context of this passage refers to *one specific method* that uses a nanoparticle solution. The passage introduces spray coating as a *potential improvement* to *that specific conventional method.*

**Actionable Improvement Strategy:**

*   **Focus on the Scope:** Be very careful to only consider information explicitly provided or directly implied within the passage. Don't bring in outside knowledge or make assumptions.
*   **Pay Attention to Specificity:** Notice words like "other," "more," "less," etc., as they often indicate comparisons. Make sure those comparisons are actually supported by the text. If a comparison isn't made in the text, then it's not the right answer.
*   **Re-read the relevant sentences:** When you're down to 2 answer choices, go back and re-read the specific parts of the passage that relate to those choices. This helps ensure you're not missing subtle implications.

Hopefully, this explanation clarifies why option A is the best answer and helps you approach similar questions with more confidence! Let me know if you have any other questions.


Perhaps the model will be more effective with FAISS vector database searches for the PDF.

# Parse PDF
The following code uses lossy conversion to turn the PDF into readable text.

Information like tables and images will be lost in the process, as demonstrated by the following snippet.

In [13]:
from PyPDF2 import PdfReader
from io import BytesIO
import requests

url = "https://www.w3.org/WAI/WCAG20/Techniques/working-examples/PDF20/table.pdf"
response = requests.get(url)
pdf_bytes = BytesIO(response.content)

text = ""
pdf_reader = PdfReader(pdf_bytes)
for page in pdf_reader.pages:
    text += page.extract_text()

print(text[0:1000])

Example table  
This is an example of a data table. 
Disability 
Category Participants  Ballots 
Completed  Ballots 
Incomplete/  
Terminated  Results  
Accuracy  Time to 
complete 
Blind  5 1 4 34.5%, n=1  1199 sec, n=1  
Low Vision  5 2 3 98.3% n=2  
(97.7%, n=3)  1716 sec, n=3  
(1934 sec, n=2)  
Dexterity  5 4 1 98.3%, n=4  1672.1 sec, n=4  
Mobility  3 3 0 95.4%, n=3  1416 sec, n=3  
 


In [14]:
# Download the PDF using requests
url = "https://satsuite.collegeboard.org/media/pdf/assessment-framework-for-digital-sat-suite.pdf"

response = requests.get(url)
pdf_bytes = BytesIO(response.content)

print(type(pdf_bytes))

text = ""
pdf_reader = PdfReader(pdf_bytes)
for page in pdf_reader.pages:
    text += page.extract_text()

print(text[0:1000])

<class '_io.BytesIO'>
Assessment Framework 
for the Digital SAT® SuiteAssessment Framework 
for the Digital SAT® Suite
Version 3.01, August 2024
About College Board
College Board reaches more than 7 million students a year, helping them 
navigate the path from high school to college and career. Our not-for-
profit membership organization was founded more than 120 years ago. 
We pioneered programs like the SAT® and AP® to expand opportunities 
for students and help them develop the skills they need. Our BigFuture® 
program helps students plan for college, pay for college, and explore 
careers. Learn more at cb.org .
Suggested Citation:  College Board. 2024. Assessment Framework for the 
Digital SAT Suite , version 3.01 (August 2024). New Y ork: College Board.
© 2024 College Board. College Board, Advanced Placement, AP , BigFuture, Landscape, Pre-AP , SAT, and 
the acorn logo are registered trademarks of College Board. AP Potential, Bluebook, Connections, PSAT, 
Skills Insight, Student S

This text data might be too big for the model to contain within the prompt.

In [15]:
client.models.count_tokens(
    model=model, contents=text
).total_tokens

153273

# Chunk the Parsed PDF
In order to use the PDF, we need to "chunk" it so bits of relevant information can be accessed at a time. In order to maximize efficiency, a vector search database will be used, as the likelihood that any key words in the question will appear on the SAT is effectively zero. 

The model will instead be a ReAct agent and figure out what to search up and call that tool as an extension and we will see what happens

do some mathplotlib thingies